## This script filtered edit sites from SAILOR; Only keeping Sailor score > 0.5 sites + <80% editing fraction as peakcaller input

In [1]:
import os
cwd = os.getcwd()

main_dir = cwd
print(main_dir)

/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks


In [2]:
from glob import glob
import pandas as pd
import os
import subprocess

input_dir = main_dir + '/sailor_output/'
filtered_sailor_bed_dir = input_dir + '/filtered_sailor_bed/'
file_paths = sorted(glob('{}*.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed'.format(input_dir)))

subprocess.run(["mkdir", filtered_sailor_bed_dir])
print(len(file_paths))

12


In [3]:
import pandas as pd
sample_id_to_label = pd.read_csv(main_dir + '/sample_id_to_label-pub.txt', sep = '\t', header = None, index_col = 0).to_dict().get(1)

print(sample_id_to_label)

{'isSTAMP10_01': 'RBFOX2 INSCRIBE PFA rep1', 'isSTAMP10_02': 'RBFOX2 INSCRIBE PFA rep2', 'isSTAMP10_03': 'RBFOX2 INSCRIBE PFA rep3', 'isSTAMP10_07': 'TDP43 INSCRIBE PFA rep1', 'isSTAMP10_08': 'TDP43 INSCRIBE PFA rep2', 'isSTAMP10_09': 'TDP43 INSCRIBE PFA rep3', 'isSTAMP10_13': 'Enzyme Only PFA rep1', 'isSTAMP10_14': 'Enzyme Only PFA rep2', 'isSTAMP10_15': 'Enzyme Only PFA rep3', 'isSTAMP10_16': 'Buffer Only PFA rep1', 'isSTAMP10_17': 'Buffer Only PFA rep2', 'isSTAMP10_18': 'Buffer Only PFA rep3'}


In [4]:
import pandas as pd
from collections import defaultdict
import numpy as np

def edit_percentage(column_edit_counts):
    return int(column_edit_counts.split(',')[0])/int(column_edit_counts.split(',')[1])

edit_info_df_dict = {}
sample_names = []


for file_path in file_paths:
    
    sample_name = file_path.split('/')[-1].split('_S')[0]
    sample_names.append(sample_name)
    print(sample_id_to_label.get(sample_name))

    file_df = pd.read_csv(file_path, sep='\t')
    print('\tOriginal edit site count:')
    print('\t\t' + str(len(file_df)))

    file_df.columns = ['chrom','start','end','score','editing','strand']

    file_df = file_df[file_df.iloc[:,3]>0.5]
    print('\tEdit site count after only keeping >0.5 Sailor score:')
    print('\t\t' + str(len(file_df)))

    file_df['edit_percentage'] = file_df.iloc[:,4].apply(edit_percentage)
    file_df = file_df[file_df['edit_percentage']<0.8]
    file_df = file_df.iloc[:, 0:6]
    print('\tEdit site count after only keeping <80% edit fraction:')
    print('\t\t' + str(len(file_df)))

    file_df.to_csv(filtered_sailor_bed_dir + sample_name + ".0.5Score0.8Fraction.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted.bam.combined.readfiltered.formatted.varfiltered.snpfiltered.ranked.bed", index = False, header=None, sep='\t')


RBFOX2 INSCRIBE PFA rep1
	Original edit site count:
		212986
	Edit site count after only keeping >0.5 Sailor score:
		177375
	Edit site count after only keeping <80% edit fraction:
		177116
RBFOX2 INSCRIBE PFA rep2
	Original edit site count:
		214155
	Edit site count after only keeping >0.5 Sailor score:
		179019
	Edit site count after only keeping <80% edit fraction:
		178797
RBFOX2 INSCRIBE PFA rep3
	Original edit site count:
		272449
	Edit site count after only keeping >0.5 Sailor score:
		225851
	Edit site count after only keeping <80% edit fraction:
		225551
TDP43 INSCRIBE PFA rep1
	Original edit site count:
		227746
	Edit site count after only keeping >0.5 Sailor score:
		184571
	Edit site count after only keeping <80% edit fraction:
		184316
TDP43 INSCRIBE PFA rep2
	Original edit site count:
		187040
	Edit site count after only keeping >0.5 Sailor score:
		154506
	Edit site count after only keeping <80% edit fraction:
		154276
TDP43 INSCRIBE PFA rep3
	Original edit site count:
	